In [ ]:
import os
os.getenv("OPENAI_API_KEY")

In [ ]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city: str):
    return f"The current weather in {city} is sunny with a temperature of 25°C."

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather],
)

# result = await Runner.run(agent, "Hello how are you? what is the weather capital of korea?")
stream = Runner.run_streamed(agent, "Hello how are you? what is the weather capital of korea?")

# print(result.final_output)
async for event in stream.stream_events():
    
    if event.type == "raw_response_event":
        continue
    elif event.type == "agent_updated_stream_event":
        print("Agent updated to", event.new_agent.name)
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type == "tool_call_output_item":
            print(event.item.output)
        elif event.item.type == "message_output_item":
            print(ItemHelpers.text_message_output(event.item))
    print("=" * 20)

Agent updated to Assistant Agent
{'arguments': '{"city":"Seoul"}', 'call_id': 'call_E3ZOHOMvYxdqM53ik63ThkWy', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_0f5c6df9d8f62a7f006925a19c9570819e8806b299b0bb1378', 'status': 'completed'}
The current weather in Seoul is sunny with a temperature of 25°C.
Hello! I'm doing well, thank you. The current weather in Seoul, the capital of South Korea, is sunny with a temperature of 25°C. If you need more details or updates for another city, feel free to ask!


In [ ]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city: str):
    return f"The current weather in {city} is sunny with a temperature of 25°C."

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather],
)

# result = await Runner.run(agent, "Hello how are you? what is the weather capital of korea?")
stream = Runner.run_streamed(agent, "Hello how are you? what is the weather capital of korea?")

message = ""
args = ""

# print(result.final_output)
async for event in stream.stream_events():
    
    if event.type == "raw_response_event":
        event_type = event.data.type
        if event_type == "response.output_text.delta":
            message += event.data.delta
            print(message)
        elif event_type == "response.function_call_arguments.delta":
            args += event.data.delta
            print(args)
        elif event_type == "response.completed":
            message = "",
            args = ""